In [ ]:
import json
vgg_file = 'C:/Users/Dell/Downloads/cat_dog_annotations.json'
with open(vgg_file, 'r') as f:
    vgg_data = json.load(f)

coco_data = {}
coco_data['images'] = []
coco_data['annotations'] = []
coco_data['categories'] = []

# Define category IDs
category_map = {}
category_id = 1

for image_id, image_data in vgg_data['_via_img_metadata'].items():
    # Add image data to COCO data
    image_info = {
        'id' : image_id + 1,
        'filename' : image_data['filename'],
        'width': image_data['size']['width'],
        'height': image_data['size']['height']
    }
    coco_data['images'].append(image_info)

    # Loop through annotations for this image
    for annotation_data in image_data['regions']:
        # Add category if necessary
        category_name = annotation_data['region_attributes']['class']
        if category_name not in category_map:
            category_info = {
                'id': category_id,
                'name': category_name
            }
            coco_data['categories'].append(category_info)
            category_map[category_name] = category_id
            category_id += 1

        # Add annotation data to COCO data
        annotation_info = {
            'id': len(coco_data['annotations']) + 1,
            'image_id': image_id + 1,
            'category_id': category_map[category_name],
            'bbox': [
                annotation_data['shape_attributes']['x'],
                annotation_data['shape_attributes']['y'],
                annotation_data['shape_attributes']['width'],
                annotation_data['shape_attributes']['height']
            ],
            'area': annotation_data['shape_attributes']['width'] * annotation_data['shape_attributes']['height'],
            'iscrowd': 0
        }
        coco_data['annotations'].append(annotation_info)

# 3. Save COCO formatted annotations to file
coco_file = 'C:/Users/Dell/Downloads/coco_file'

with open(coco_file, 'w') as f:
    json.dump(coco_data, f)


# 2. Load that image

In [ ]:
import json
from pycocotools.coco import COCO
import random
from PIL import Image, ImageEnhance, ImageOps
import numpy as np


vgg_file = 'C:/Users/Dell/Downloads/coco_file'
image_path = 'C:/Users/Dell/Downloads/cat_dog_images'

with open(vgg_file, 'r') as f:
    vgg_data = json.load(f)

image_data = vgg_data[image_path]

# 2. Load the image
image = Image.open(image_path)

# 3. Resize the image

In [ ]:
resize_scale = random.uniform(0.6, 1.0)
new_width = int(image.width * resize_scale)
new_height = int(image.height * resize_scale)
resized_image = image.resize((new_width, new_height))

# Adjust bounding boxes according to new size
for annotation_data in image_data['regions']:
    annotation_data['shape_attributes']['x'] *= resize_scale
    annotation_data['shape_attributes']['y'] *= resize_scale
    annotation_data['shape_attributes']['width'] *= resize_scale
    annotation_data['shape_attributes']['height'] *= resize_scale

# 4. Add random brightness to the resized image

In [ ]:
brightness_factor = random.uniform(0.7, 1.3)
enhancer = ImageEnhance.Brightness(resized_image)
brightened_image = enhancer.enhance(brightness_factor)

# 5. Rotate the brighter image

In [ ]:
rotation_angle = random.uniform(0, 360)
rotated_image = brightened_image.rotate(rotation_angle, resample=Image.BICUBIC, expand=True)

# Adjust bounding boxes according to rotation
for annotation_data in image_data['regions']:
    x, y, w, h = annotation_data['shape_attributes']['x'], annotation_data['shape_attributes']['y'], annotation_data['shape_attributes']['width'], annotation_data['shape_attributes']['height']
    corners = np.array([[x,y], [x+w,y], [x+w,y+h], [x,y+h]])
    center = np.array([x + w / 2, y + h / 2])
    rotation_matrix = np.array([[np.cos(rotation_angle), -np.sin(rotation_angle)], [np.sin(rotation_angle), np.cos(rotation_angle)]])
    new_corners = (np.matmul((corners - center), rotation_matrix) + center).tolist()
    new_x = min([corner[0] for corner in new_corners])
    new_y = min([corner[1] for corner in new_corners])
    new_w = max([corner[0] for corner in new_corners]) - new_x
    new_h = max([corner[1] for corner in new_corners]) - new_y
    annotation_data['shape_attributes']['x'] = new_x
    annotation_data['shape_attributes']['y'] = new_y
    annotation_data['shape_attributes']['width'] = new_w
    annotation_data['shape_attributes']['height'] = new_h


# 6. Save the resulting image on disk

In [ ]:

resulting_image_path = 'path/to/save/resulting/image'
rotated_image.save(resulting_image_path)

# You can also save the updated VGG data as a new JSON file if you want to use it for training later
updated_vgg_file = 'path/to/save/updated/vgg/file'
with open(updated_vgg_file, 'w') as f:
    json.dump(vgg_data, f)